In [31]:
class State:
    def __init__(self, size=3):
        self.size = size
        self.chessboard = [[None for _ in range(size)] for _ in range(size)]
        self.winner = self.judge()
        self.depth = None

    def judge(self):
        # 检查行
        for row in self.chessboard:
            if row[0] is not None and all(cell == row[0] for cell in row):
                return row[0]

        # 检查列
        for col in range(self.size):
            if (
                self.chessboard[0][col] is not None
                and all(self.chessboard[row][col] == self.chessboard[0][col] for row in range(self.size))
            ):
                return self.chessboard[0][col]

        # 检查主对角线
        if (
            self.chessboard[0][0] is not None
            and all(self.chessboard[i][i] == self.chessboard[0][0] for i in range(self.size))
        ):
            return self.chessboard[0][0]

        # 检查副对角线
        if (
            self.chessboard[0][self.size - 1] is not None
            and all(
                self.chessboard[i][self.size - 1 - i] == self.chessboard[0][self.size - 1]
                for i in range(self.size)
            )
        ):
            return self.chessboard[0][self.size - 1]

        return None

    def drop(self, color, loc):
        
        x, y = loc
        if self.chessboard[x][y] is None:
            self.chessboard[x][y] = color
            self.winner = self.judge()
        else:
            raise ValueError(f"Position {loc} is already occupied!")

    def __eq__(self, other):
        if not isinstance(other, State):
            return False
        return self.chessboard == other.chessboard

    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.chessboard))

    def print_board(self):
        for row in self.chessboard:
            print(" | ".join(cell if cell else "." for cell in row))


class Chequer:
    def __init__(self, color, loc):
        self.color = color
        self.loc = loc

    def __eq__(self, other):
        if not isinstance(other, Chequer):
            return False
        return self.color == other.color and self.loc == other.loc

    def __hash__(self):
        return hash((self.color, self.loc))


In [32]:
state = State()
state.drop("X", (0, 0))
state.drop("O", (1, 1))
state.print_board()

print("Winner:", state.winner)
chess = Chequer("X", (0, 0))
print("Chequer hash:", hash(chess))


X | . | .
. | O | .
. | . | .
Winner: None
Chequer hash: -8420385017634439078


In [33]:
class Robot:
    def __init__(self, color):
        self.color = color  # 机器人的棋子颜色

    def next(self, state, maxdepth):
        best_move = None
        best_score = float("-inf")

        for move in self.get_available_moves(state):
            new_state = self.simulate_move(state, move, self.color)
            score = self.minimax(new_state, maxdepth - 1, False)
            if score > best_score:
                best_score = score
                best_move = move

        return best_move

    def minimax(self, state, depth, is_maximizing):

        if depth == 0 or state.winner is not None:
            return self.evaluate(state)

        if is_maximizing:
            max_eval = float("-inf")
            for move in self.get_available_moves(state):
                new_state = self.simulate_move(state, move, self.color)
                eval = self.minimax(new_state, depth - 1, False)
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float("inf")
            opponent_color = "X" if self.color == "O" else "O"
            for move in self.get_available_moves(state):
                new_state = self.simulate_move(state, move, opponent_color)
                eval = self.minimax(new_state, depth - 1, True)
                min_eval = min(min_eval, eval)
            return min_eval

    def evaluate(self, state):

        def count_winning_lines(color):
            count = 0
            # 检查行
            for row in state.chessboard:
                if all(cell == color or cell is None for cell in row):
                    count += 1

            # 检查列
            for col in range(state.size):
                if all(state.chessboard[row][col] == color or state.chessboard[row][col] is None for row in range(state.size)):
                    count += 1

            # 检查主对角线
            if all(state.chessboard[i][i] == color or state.chessboard[i][i] is None for i in range(state.size)):
                count += 1

            # 检查副对角线
            if all(state.chessboard[i][state.size - 1 - i] == color or state.chessboard[i][state.size - 1 - i] is None for i in range(state.size)):
                count += 1

            return count

        return count_winning_lines(self.color) - count_winning_lines("X" if self.color == "O" else "O")

    def get_available_moves(self, state):

        return [(x, y) for x in range(state.size) for y in range(state.size) if state.chessboard[x][y] is None]

    def simulate_move(self, state, move, color):

        new_state = State(state.size)
        new_state.chessboard = [row[:] for row in state.chessboard]
        new_state.drop(color, move)
        return new_state


In [34]:
def main():
    # 初始化棋盘
    board_size = 3
    state = State(size=board_size)

    # 玩家选择颜色
    player_color = input("请输入你的颜色 (O 或 X): ").strip().upper()
    while player_color not in ("O", "X"):
        player_color = input("无效输入，请重新输入你的颜色 (O 或 X): ").strip().upper()

    robot_color = "X" if player_color == "O" else "O"
    robot_player = Robot(robot_color)

    print(f"你选择了 {player_color}，机器人是 {robot_color}。")

    
    turn = "X"  # X先手
    while state.winner is None and any(None in row for row in state.chessboard):
        state.print_board()
        print(f"当前轮到 {'你' if turn == player_color else '机器人'} ({turn})")

        if turn == player_color:
            # 玩家回合
            while True:
                try:
                    move_input = input("请输入你的落子位置(如 1 1): ")
                    x, y = map(int, move_input.split())
                    if state.chessboard[x][y] is not None:
                        print("该位置已被占用，请重新输入。")
                        continue
                    state.drop(player_color, (x, y))
                    break
                except (ValueError, IndexError):
                    print("输入格式错误，请重新输入。")

        else:

            next_move = robot_player.next(state, maxdepth=3)
            print(f"机器人选择了位置: {next_move}")
            state.drop(robot_color, next_move)

        # 检查胜利或平局状态
        if state.winner is not None:
            print(f"游戏结束，胜者是: {state.winner}")
            break
        elif not any(None in row for row in state.chessboard):
            print("游戏结束，平局！")
            break

        # 切换回合
        turn = "O" if turn == "X" else "X"

    # 输出最终棋盘
    print("最终棋盘:")
    state.print_board()

if __name__ == "__main__":
    main()



你选择了 O，机器人是 X。
. | . | .
. | . | .
. | . | .
当前轮到 机器人 (X)
机器人选择了位置: (1, 1)
. | . | .
. | X | .
. | . | .
当前轮到 你 (O)
O | . | .
. | X | .
. | . | .
当前轮到 机器人 (X)
机器人选择了位置: (0, 1)
O | X | .
. | X | .
. | . | .
当前轮到 你 (O)
输入格式错误，请重新输入。
输入格式错误，请重新输入。
输入格式错误，请重新输入。
O | X | .
. | X | .
. | O | .
当前轮到 机器人 (X)
机器人选择了位置: (0, 2)
O | X | X
. | X | .
. | O | .
当前轮到 你 (O)
O | X | X
. | X | .
. | O | O
当前轮到 机器人 (X)
机器人选择了位置: (2, 0)
游戏结束，胜者是: X
最终棋盘:
O | X | X
. | X | .
X | O | O
